### Apprentissage 

In [14]:
# Imports : keras 2.6.1 / tensorflow / theano / ViennaRNA package python (installer via 'conda install viennarna')

import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers.convolutional import AveragePooling2D
from keras.callbacks import EarlyStopping
import RNA
from sklearn.metrics import roc_curve, auc
import numpy as np
import random
import gzip
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse
import pandas as pd
import csv

from random import shuffle

#### Chargement du réseau, sans apprentissage, car les poids sont enregistrés


In [3]:
# paramètres :

# paramètres 
true_y = Data[5]
training = Data[0]
y = Data[1]
batch_size=48
epochs=25
verbose1 = 1
verbose2 = 1
validation = Data[2]
val_y = Data[3]
testing = Data[4]
matrixsize11 = 6
nbfilter11 = 24
matrixsize12 = 7
nbfilter12 = 4
matrixsize21 = 6
nbfilter21 = 24
matrixsize22 = 7
nbfilter22 = 4
nbfilter1 = 64
kernelsize = 7
nbfilters2 = 64
kernel_size2 = 7
Dense1 = 128
Dense2 = 512
Dense3 = 512
Dense4 = 128
Dense5 = 0

k = matrixsize11
# init_weights
I = np.eye(k)
M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
I2 = np.zeros((k,k))
M2 = np.zeros((k,k))
for j in range(k):
    I2[:,j] = I[:,k-j-1]
    M2[:,j] = M[:,k-j-1]        
W = np.zeros((k,k,1,nbfilter11))
W[:,:,0,0] = I
W[:,:,0,1] = I2
W[:,:,0,2] = M
W[:,:,0,3] = M2
for j in range(4,nbfilter11):
    W[:,:,0,j] = np.random.randn(k,k)*0.2


k2 = matrixsize12
I = np.eye(k2)
M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
I2=np.zeros((k2,k2))
M2=np.zeros((k2,k2))
for j in range(k2):
    I2[:,j] = I[:,k2-j-1]
    M2[:,j] = M[:,k2-j-1]   

Z = np.zeros((k2,k2,nbfilter11,nbfilter12))

for u in range(nbfilter12):
    Z[:,:,u,0] = I
    Z[:,:,u,1] = I2
    Z[:,:,u,2] = M
    Z[:,:,u,3] = M2
    for p in range(4,nbfilter12):
        Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

c2d1_input = keras.Input(shape=(36,36,1))
cnn2d1 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d1_input)
cnn2d1 = AveragePooling2D(pool_size=(3,3))(cnn2d1)
cnn2d1 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d1)
cnn2d1 = Dropout(0.2)(cnn2d1)
cnn2d1 = Flatten()(cnn2d1)

c2d2_input = keras.Input(shape=(36,36,1))
cnn2d2 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d2_input)
cnn2d2 = AveragePooling2D(pool_size=(3,3))(cnn2d2)
cnn2d2 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d2)
cnn2d2 = Dropout(0.2)(cnn2d2)
cnn2d2 = Flatten()(cnn2d2)

c1d1_input = keras.Input(shape=(36,4))
cnn1d1 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d1_input)
cnn1d1 = MaxPooling1D(pool_size=8)(cnn1d1)
cnn1d1 = Dropout(0.2)(cnn1d1)
cnn1d1 = Flatten()(cnn1d1)

c1d2_input = keras.Input(shape=(36,4))
cnn1d2 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d2_input)
cnn1d2 = MaxPooling1D(pool_size=8)(cnn1d2)
cnn1d2 = Dropout(0.2)(cnn1d2)
cnn1d2 = Flatten()(cnn1d2)

c1d3_input = keras.Input(shape=(36,4))
cnn1d3 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d3_input)
cnn1d3 = MaxPooling1D(pool_size=8)(cnn1d3)
cnn1d3 = Dropout(0.2)(cnn1d3)
cnn1d3 = Flatten()(cnn1d3)

c1d4_input = keras.Input(shape=(36,4))
cnn1d4 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d4_input)
cnn1d4 = MaxPooling1D(pool_size=8)(cnn1d4)
cnn1d4 = Dropout(0.2)(cnn1d4)
cnn1d4 = Flatten()(cnn1d4)

model21 = keras.layers.concatenate([cnn1d1,cnn1d2])
model21 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model21)
model21 = BatchNormalization()(model21)
model21 = Activation('relu')(model21)
model21 = Dropout(0.3)(model21)

model22 = keras.layers.concatenate([cnn1d3,cnn1d4])
model22 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model22)
model22 = BatchNormalization()(model22)
model22 = Activation('relu')(model22)
model22 = Dropout(0.3)(model22)

model2 = keras.layers.concatenate([model21,model22])
model2 = Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model2)
model2 = BatchNormalization()(model2)
model2 = Activation('relu')(model2)
model2 = Dropout(0.3)(model2)

model1 = keras.layers.concatenate([cnn2d1,cnn2d2])
model1 = Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model1)
model1 = Dropout(0.1)(model1)
model1 = BatchNormalization()(model1)
model1 = Activation('relu')(model1)

model = keras.layers.concatenate([model1,model2])
model = Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.2)(model)
model = Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.2)(model)

if Dense5>0:
    model = Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Dropout(0.2)(model)

model = Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
interaction_output = Activation('softmax')(model)

interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])

interaction.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#### Chargement des poids du réseau : 

In [5]:
interaction.load_weights("model_rna-rna_deep_livrable.h5")

### Test benchmark

#### Chargement des données benchmark

In [253]:
datatest = pd.read_csv("genomes/benchmark_interactionsv2.csv", sep = "\t",header=None)
datatest = np.array(datatest)
sequence1=[]
sequence2=[]
longueur1=[]
longueur2=[]
l = len(datatest)
for i in range(l):
    data = datatest[i][0].split(' ')
    data[0] = data[0].split('[')
    data[0][1] = data[0][1].split('\n')
    data[1] = data[1].split('\n')
    seq1=data[0][1][0][1:-1]
    seq2=data[1][0][1:-1]
    len1=int(data[2])
    len2=int(data[3])
    sequence1.append(seq1)
    sequence2.append(seq2)
    longueur1.append(len1)
    longueur2.append(len2)
sequence1=np.array(sequence1,dtype=None,ndmin=2)
sequence2=np.array(sequence2,dtype=None,ndmin=2)
longueur1=np.array(longueur1,dtype=None,ndmin=2)
longueur2=np.array(longueur2,dtype=None,ndmin=2)

bdd = np.zeros((len(datatest),4))
bdd = np.array(bdd,dtype='<U206',copy=True)
bdd[:,0]=sequence1
bdd[:,1]=sequence2
bdd[:,2]=longueur1
bdd[:,3]=longueur2

Fonctions de remplissage des inputs en fonction de la longueur du tronçon de séquence souhaité : 
longueur : paramètre f (décidé à 25)

In [32]:
def remplissage(matrice,f):
    matrice_out = np.zeros((1,36,36,1))
    matrice_out[0,:f,:f,0] = matrice[0,:f,:f,0]
    return matrice_out
def remplissage_seq(matrice,f):
    matrice_out = np.zeros((1,36,4))
    matrice_out[0,:f] = matrice[0,:f]
    return matrice_out

Test de deep_rna_rna sur les données benchmark : fenêtrage  de vitesse s=12 sur les deux séquences :
- la fenêtre mesure 25 nucléotides (paramètre f)
- deep rna rna prédit pour chaue couple de fenêtres si ces deux sites sont suceptibles d'interagir ensemble

Le résultat est donc une matrice, dont on ne garde que les valeurs supérieures au seuil.
Les coordonnées correspondant à ces valeurs sont considérées comme le début de sites et les nucléotides correspondants sont classifiés comme engagés. Cela est comparé aux labels des nucléotides réellement engagés dans des interactions parmi les deux séquences.

In [370]:
# f longueur des sites à tester sur chaque séquence
f = 25
# s vitesse de glissement de la fenêtre de test le long des deux séquences
s = 12
# facteur_seuil afin de décider quelles prédictions sont considérées comme des vraies interactions
facteur_seuil = 0.95
#calcul des MCC sur la base de données benchmark constituées de 2433 paires de séquences dont les milieux interagissent ensemble
mcc = np.zeros(len(bdd))

for r in range(len(bdd)):
    
    seq1 = bdd[r,0]
    seq2 = bdd[r,1]
    dim = len(seq1)
    dim2 = len(seq2)
    matrice = np.zeros((1,dim,dim2,1)) 
    matrice1 = np.zeros((1,dim,dim2,1))     
    matrice2 = np.zeros((1,dim,4))
    matrice3 = np.zeros((1,dim2,4))

    proba1 = np.array(RNA.pfl_fold_up(seq1,1,dim,dim))[:,1]
    proba2 = np.array(RNA.pfl_fold_up(seq2,1,dim2,dim2))[:,1]

    for j in range(dim):
        for k in range(dim2):
            if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                matrice[0,j,k,0] = 1
            elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                matrice[0,j,k,0] = 1
            elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                matrice[0,j,k,0] = 1
            matrice1[0,j,k,0] = proba1[j] + proba2[k]
    for j in range(dim):
        if seq1[j]=='a':
            matrice2[0,j,0] = 1
        elif seq1[j]=='u':
            matrice2[0,j,1] = 1
        elif seq1[j]=='g':
            matrice2[0,j,2] = 1
        elif seq1[j]=='c':
            matrice2[0,j,3] = 1
    for j in range(dim2):
        if seq2[j]=='a':
            matrice3[0,j,0] = 1
        elif seq2[j]=='u':
            matrice3[0,j,1] = 1
        elif seq2[j]=='g':
            matrice3[0,j,2] = 1
        elif seq2[j]=='c':
            matrice3[0,j,3] = 1
    u = int((dim-f+1)/s)
    u2 = int((dim2-f+1)/s)
    resultats = np.zeros((u,u2))

    for i in range(u):
        for j in range(u2):        
            test = []
            test.append(remplissage(matrice[:,i*s:i*s+36,j:j+36,:],f))
            test.append(remplissage(matrice1[:,i*s:i*s+36,j:j+36,:],f))            
            test.append(remplissage_seq(matrice2[:,i*s:i*s+36],f))
            rev = np.zeros((1,36,4))
            for q in range(36):
                rev[:,36-q-1,:] = matrice2[:,q,:]
            test.append(remplissage_seq(rev,f))
            test.append(remplissage_seq(matrice3[:,j*s:j*s+36],f))
            rev2 = np.zeros((1,36,4))
            for q in range(36):
                rev2[:,36-q-1,:] = matrice3[:,q,:]
            test.append(remplissage_seq(rev2,f))

            resultats[i,j] = interaction.predict(test)[:,1]
    # seuil :
    seuil = np.max(resultats)*facteur_seuil
    pred = np.zeros(dim+dim2)
    for i in range(u):
        for j in range(u2):
            if resultats[i,j]>seuil:
                pred[i*s:i*s+f] = np.ones(f)
                pred[j*s:j*s+f] = np.ones(f)
    label = np.zeros(dim+dim2)
    label[85:85+int(bdd[r,2])]=np.ones(int(bdd[r,2]))
    label[dim+85:dim+85+int(bdd[r,3])]=np.ones(int(bdd[r,3]))
    # MCC : 
    mcc[r] = calculate_performance(dim+dim2,pred,label)[4]

In [ ]:
np.save('MCC_benchmark_deep_rna_rna_.npy', mcc)

Les MCC sont enregistrés dans le fichier 'MCC_benchmark_deep_rna_rna_.npy'

In [16]:
mcc = np.load('MCC_benchmark_deep_rna_rna_.npy')

In [19]:
np.mean(mcc), sqrt(np.var(mcc)), np.min(mcc), np.max(mcc)

(0.08009286560875584,
 0.17274048708220247,
 -0.30744512576786553,
 0.7226200938888836)

In [7]:
def plot_performances():
    
    mcc = np.load('MCC_benchmark_deep_rna_rna_.npy')
    length = len(mcc)    
    x = np.linspace(1,length,length)
    
    #Print the MCC and its average value
    fig1 = plt.figure(1)
    plt.clf()
    plt.title("MCC")
    p8 = plt.plot(x,mcc, label='MCC',marker='v')
    avg_mcc = np.mean(mcc)
    y = np.ones(length)
    y = np.dot(avg_mcc,y)
    p9 = plt.plot(x,y,marker='.')
    fig1.savefig('benchmark_result/'+'Deep_rna_rna'+'_MCC.png')
    plt.show()
    
    print("Average MCC : ", avg_mcc)

In [ ]:
plot_performances()